In [ ]:
from DeepFMKit.core import DeepFitFramework
from DeepFMKit.plotting import default_rc
from DeepFMKit.experiments import Experiment
import numpy as np

import matplotlib.pyplot as plt
plt.rcParams.update(default_rc)

In [ ]:
dff = DeepFitFramework(raw_file='../test/raw_data.txt', raw_labels=['raw1'])
dff.fit('raw1', n=20)

In [ ]:
axes = dff.plot()
plt.show()

In [ ]:
ax = dff.plot(which='m', figsize=(5,2), lw=1, ls='--', c='tomato')
ax.legend()
plt.show()

In [ ]:
ax = dff.raws['raw1'].plot(figsize=(5,3), xrange=[0,0.01], c='lime')
ax.grid(False)
plt.show()